In [2]:
from torch import tensor
import torch
from sklearn.cluster import KMeans
import numpy as np

agents_data = [tensor([[-0.0075,  0.5364]]), tensor([[-0.8230, -0.7359]]), tensor([[-0.3852,  0.2682]]), tensor([[-0.0198,  0.7929]])]
# targets_data = [tensor([[-0.0887,  0.2646]]), tensor([[-0.3022, -0.1966]]), tensor([[0.3953, 0.6000]]), tensor([[ 0.9055, -0.9277]])]
targets_data = [tensor([[-0.0887,  0.2646]]), tensor([[-0.3022, -0.1966]]), tensor([[0.3953, 0.6000]]), tensor([[ 0.9055, -0.9277]]), 
                tensor([[-0.6058, -0.0803]]), tensor([[0.2823, 0.4182]]), tensor([[-0.5896,  0.6298]]), tensor([[0.6618, 0.7357]]), 
                tensor([[-0.2421, -0.7303]]), tensor([[ 0.4424, -0.0980]]), tensor([[-0.2639,  0.2779]]), tensor([[ 0.6133, -0.0806]])]

In [6]:
import torch
import numpy as np
from sklearn.cluster import KMeans

# 에이전트와 타겟 클래스 정의 (간단한 예제)
class Agent:
    def __init__(self, x, y):
        self.state = type('', (), {})()  # 간단한 객체 속성 생성
        self.state.pos = torch.tensor([[x, y]], dtype=torch.float32)

class Target:
    def __init__(self, x, y):
        self.state = type('', (), {})()
        self.state.pos = torch.tensor([[x, y]], dtype=torch.float32)

def targets_grouping(agents, targets):
    """
    agents: 에이전트 객체 리스트 (각 객체는 state.pos 속성에 위치 좌표를 가짐)
    targets: 타겟 객체 리스트 (각 객체는 state.pos 속성에 위치 좌표를 가짐)

    반환값: {에이전트 객체: 해당 에이전트에게 할당된 타겟 객체 리스트}
    """
    n_agents = len(agents)
    n_targets = len(targets)

    # 1. 타겟 좌표 추출
    target_positions = torch.cat([target.state.pos for target in targets], dim=0).numpy()  # shape: (n_targets, 2)

    # 2. KMeans로 타겟을 클러스터링 (n_agents개의 클러스터)
    kmeans = KMeans(n_clusters=n_agents, random_state=0, n_init=10)
    kmeans.fit(target_positions)
    labels = kmeans.labels_  # 각 타겟에 대한 클러스터 할당 (0 ~ n_agents-1)

    # 3. 클러스터별 타겟 그룹화
    clusters = {i: [] for i in range(n_agents)}
    for label, target in zip(labels, targets):
        clusters[label].append(target)  # 타겟 객체 자체를 저장

    # 4. 에이전트 위치 추출
    agent_positions = np.array([agent.state.pos.numpy().flatten() for agent in agents])  # shape: (n_agents, 2)

    # 5. 에이전트와 클러스터를 매칭 (각 에이전트에게 가장 가까운 클러스터 배정)
    centroids = kmeans.cluster_centers_  # shape: (n_agents, 2)
    assigned_clusters = list(range(n_agents))  # 배정되지 않은 클러스터 인덱스 리스트
    agent_assignments = {}

    remaining_agents = set(agents)  # 아직 클러스터가 배정되지 않은 에이전트 집합

    while remaining_agents and assigned_clusters:
        for agent in list(remaining_agents):
            agent_idx = agents.index(agent)
            distances = np.linalg.norm(centroids - agent_positions[agent_idx], axis=1)

            # 아직 배정되지 않은 클러스터 중 가장 가까운 것을 선택
            closest_cluster = assigned_clusters[np.argmin([distances[i] for i in assigned_clusters])]

            # 해당 클러스터를 에이전트에게 배정
            agent_assignments[agent] = clusters[closest_cluster]

            # 배정된 클러스터 제거
            assigned_clusters.remove(closest_cluster)
            remaining_agents.remove(agent)

    return agent_assignments

# 6. 샘플 데이터 생성
agents = [Agent(-0.5, 0.5), Agent(0.7, -0.3), Agent(-0.2, -0.8), Agent(0.3, 0.9)]
targets = [
    Target(-0.7, 0.2), Target(-0.3, -0.5), Target(0.8, -0.4), Target(0.1, 0.7),
    Target(-0.6, 0.8), Target(0.5, -0.9), Target(0.3, 0.4), Target(0.9, 0.2),
    Target(-0.8, -0.6), Target(-0.1, -0.2), Target(0.7, 0.6), Target(-0.4, 0.3)
]

# 7. 그룹화 실행
assigned_targets = targets_grouping(agents, targets)

# 8. 결과 출력
for agent, target_list in assigned_targets.items():
    print(f"Agent at {agent.state.pos.numpy()} -> Assigned {len(target_list)} targets:")
    for target in target_list:
        print(f"    Target at {target.state.pos.numpy()}")


Agent at [[ 0.7 -0.3]] -> Assigned 2 targets:
    Target at [[ 0.8 -0.4]]
    Target at [[ 0.5 -0.9]]
Agent at [[-0.5  0.5]] -> Assigned 3 targets:
    Target at [[-0.7  0.2]]
    Target at [[-0.6  0.8]]
    Target at [[-0.4  0.3]]
Agent at [[0.3 0.9]] -> Assigned 4 targets:
    Target at [[0.1 0.7]]
    Target at [[0.3 0.4]]
    Target at [[0.9 0.2]]
    Target at [[0.7 0.6]]
Agent at [[-0.2 -0.8]] -> Assigned 3 targets:
    Target at [[-0.3 -0.5]]
    Target at [[-0.8 -0.6]]
    Target at [[-0.1 -0.2]]


In [7]:
import torch
import numpy as np
from sklearn.cluster import KMeans

def targets_grouping(agents, targets):
    """
    agents: 에이전트 객체 리스트 (각 객체는 state.pos 속성에 위치 좌표를 가짐)
    targets: 타겟 객체 리스트 (각 객체는 state.pos 속성에 위치 좌표를 가짐)

    반환값: 각 에이전트가 방문할 타겟 리스트의 이차원 배열 (List[List[Target]])
    """
    n_agents = len(agents)
    n_targets = len(targets)

    # 1. 타겟 좌표 추출
    target_positions = torch.cat([target.state.pos for target in targets], dim=0).numpy()  # shape: (n_targets, 2)

    # 2. KMeans로 타겟을 클러스터링 (n_agents개의 클러스터)
    kmeans = KMeans(n_clusters=n_agents, random_state=0, n_init=10)
    kmeans.fit(target_positions)
    labels = kmeans.labels_  # 각 타겟에 대한 클러스터 할당 (0 ~ n_agents-1)

    # 3. 클러스터별 타겟 그룹화
    clusters = {i: [] for i in range(n_agents)}
    for label, target in zip(labels, targets):
        clusters[label].append(target)  # 타겟 객체 자체를 저장

    # 4. 에이전트 위치 추출
    agent_positions = np.array([agent.state.pos.numpy().flatten() for agent in agents])  # shape: (n_agents, 2)

    # 5. 에이전트와 클러스터를 매칭 (각 에이전트에게 가장 가까운 클러스터 배정)
    centroids = kmeans.cluster_centers_  # shape: (n_agents, 2)
    assigned_clusters = list(range(n_agents))  # 배정되지 않은 클러스터 인덱스 리스트
    agent_assignments = [None] * n_agents  # 최종 리턴 리스트

    remaining_agents = set(agents)  # 아직 클러스터가 배정되지 않은 에이전트 집합

    while remaining_agents and assigned_clusters:
        for agent in list(remaining_agents):
            agent_idx = agents.index(agent)
            distances = np.linalg.norm(centroids - agent_positions[agent_idx], axis=1)

            # 아직 배정되지 않은 클러스터 중 가장 가까운 것을 선택
            closest_cluster = assigned_clusters[np.argmin([distances[i] for i in assigned_clusters])]

            # 해당 클러스터를 에이전트에게 배정
            agent_assignments[agent_idx] = clusters[closest_cluster]

            # 배정된 클러스터 제거
            assigned_clusters.remove(closest_cluster)
            remaining_agents.remove(agent)

    return agent_assignments  # 최종 이차원 리스트

print(targets_grouping(agents_data, targets_data))

AttributeError: 'Tensor' object has no attribute 'state'